In [16]:
import numpy as np
from scipy.optimize import root
import prop
import math as m

G1 = 298
T11 = 50
T12 = 37
G2 = 298
T21 = 32
P1 = 0.243
P2 = 3.3
HOT = 'REFPROP::R236ea'
COLD = 'REFPROP::R236ea'


Q = G1*(prop.t_p(T11,P1, COLD)["H"]-prop.t_p(T12,P1, COLD)["H"])
T22 = prop.h_p(  prop.t_p(T21,P2, COLD)["H"] + Q/G2,P2, COLD)["T"]


Dvnes=0.01
DeltaTube=0.001
Dvnut=Dvnes-2*DeltaTube
step=2.5*Dvnes        
Hchan=0.25*Dvnes
Deltaplate = 0.0005
lambdaw = 20
etta=1
w1 = 23
w2 = 3
Z = 8

T1av = (T11+T12)/2
T2av = (T21+T22)/2

ro1 = prop.t_p(T1av,P1, HOT)['ro']
ro2 = prop.t_p(T2av,P2, COLD)['ro']
nu1 = prop.t_p(T1av,P1, HOT)['nu']
nu2 = prop.t_p(T2av,P2, COLD)['nu']
lambda1 = prop.t_p(T1av,P1, HOT)['lamda']
lambda2 = prop.t_p(T2av,P2, COLD)['lamda']
Pr1 = prop.t_p(T1av,P1, HOT)['Pr']
Pr2 = prop.t_p(T2av,P2, COLD)['Pr']

dTmax = T11 - T22
dTmin = T12 - T21
dT = (dTmax - dTmin) / (np.log(dTmax / dTmin))

Re2 = w2*Dvnut/nu2
Nu2 = 0.021*Re2**0.8*Pr2**0.43
def func(x):
    delta = 10**-5
    left = x
    right = 1/(2*np.log10((2.51/(Re2*(x)**(1/2)))+(delta/3.7)))**2
    return left-right
sol = root(func, 0.01)
f2=float(sol.x)
alpha2 = Nu2*lambda2/Dvnut
Re1 = w1*Dvnes/nu1
Nu1 = 0.195*Re2**0.65*Pr1**(-1/3)
f1 = 1.7*Re1**(-0.5)
alpha1 = Nu1*lambda1/Dvnes
KF = Q*1000/dT

F_vneshn = 4*(((2**0.5)*step/2)**2 - np.pi*(Dvnes**2)/4) + 2*Hchan* np.pi*Dvnes
F_vntr = np.pi*(Dvnes-2*DeltaTube)*2*(Hchan+Deltaplate)
F_trube = np.pi*Dvnes*2*(Hchan+Deltaplate)
koef1 = F_vntr/F_vneshn
koef2 = F_vneshn / F_trube

def func(x):
    Fvnes=x
    a1 = 1/KF
    a2 = 1/(etta*alpha1*Fvnes)
    a3 = ((1/2)*(np.log(Dvnes/Dvnut))*(Dvnes/(lambdaw*(Fvnes/koef2))))
    a4 = 1/(alpha2*Fvnes*koef1)
    return a1-a2-a3-a4
sol = root(func, 0.002)
x=sol.x
Fvnes=float(x)
Nsumm = m.ceil(Fvnes/(F_vneshn)/Z)

S2 = G2/(ro2*w2)
S1 = G1/(ro1*w1)


NH =  m.ceil(Nsumm*(np.pi*Dvnut**2)/(S2*4))
NW =  m.ceil(S1/(4*NH*Hchan*(step-(Dvnut+2*DeltaTube))))
NL =  m.ceil(4*S2/(NW*np.pi*Dvnut**2))
S1_y = 4*NW*NH*Hchan*(step-(Dvnut+2*DeltaTube))
S2_y = NW*NL*np.pi*Dvnut**2/4


w1_y = G1/(ro1*S1_y)
w2_y = G2/(ro2*S2_y)
Nsumm_y = NH*NW*NL
Fvnes_y =Nsumm_y*F_vneshn*Z

W=round(NW*step, 4)
H=round(NH*((Hchan+Deltaplate)*2), 4)
L=round(NL*step*Z, 4)

dP1_y = f1*Z*L/Dvnes*ro1*w1_y**2/2
dP2_y = f2*Z*H/Dvnut*ro2*w2_y**2/2
    
Ntube = NW*NL*Z
Nplate = NH*2*Z
Ltube = 2* (Hchan+Deltaplate)*NH
Wplate = NW*step
Lplate = NL*step

V = np.pi*(Dvnes**2-Dvnut**2)/4*Ltube*Ntube + Lplate*Wplate*Deltaplate*Nplate
ro_stal = 7832 # кг/м3
stoimost_metal = 300 # руб за кг

Massa =   ro_stal * V 
Massa_tube  = np.pi*(Dvnes**2-Dvnut**2)/4*Ltube*Ntube * ro_stal
Massa_plate = Lplate*Wplate*Deltaplate*Nplate * ro_stal

Stoimost_tube = stoimost_metal*Massa_tube
Stoimost_plate = stoimost_metal*Massa_plate
Stoimost_TO = Stoimost_tube +Stoimost_plate 
Svarka = 75*Nplate
Korpys = 300000*V
money = round(Stoimost_tube + Stoimost_plate + Svarka + Korpys)

print('Длина ТО:', L)
print('Ширина ТО:', W)
print('Высота ТО:', H)
print('Площадь ТО:', Fvnes_y)
print('Потери давления по горячему каналу:',dP1_y)
print('Потери давления по холодному каналу:',dP2_y)
print('Стоимость ТО:',money)

Длина ТО: 8.0
Ширина ТО: 0.9
Высота ТО: 0.978
Площадь ТО: 2052.2421489397625
Потери давления по горячему каналу: 75580.9496730762
Потери давления по холодному каналу: 107343.70726940277
Стоимость ТО: 4149213
